In [ ]:
全てのラベルを1に割り振ってしまう、という状態。
過学習しない・量的変数を増やす事を目標にする

仮説：
木のダメージにまつわる本質的な情報を取り出せると精度が向上する。
歩道に生えているか、管理者は何人か、管理している人は専門家かボランティアか
ダメージに関係しそうな木の問題はあるか、木の種類は何か、
行政区分＝木の管理に割かれている予算はどれほどか、といった情報が抽出できると
精度が向上するはず。

これまでに試した分析法
- ["curb_loc","steward","guards","sidewalk","user_type"]をダミー化する(drop_first=False)
- "problems"をダミー化する。その際最初の10行だけを残す

20231213、今日試す分析法
- spc_commonをダミー化。その際最初の15行だけを残す→効果なし
- problems、spc＿commonを量的変数を出現頻度を表す量的変数にする→効果なし

- borocodeをダミー化

# - stewardを量的変数に戻す。1.5, 3.5, 5の3値にする
boro_ct ニューヨーク市のセンサストラクト（小地域統計区域）のコード
cb_num コミュニティボード番号
st_senate 州上院地区番号
st_assem 州議会地区番号
cncldist 市議会の地区番号
→行政区分をダミー化、もしくは頻度の量的変数にする

→ ダミー化は全く効かない。代わりにこれまでダミー化した値すべてをターゲットエンコーディングに切り替える。

Target Encodingの変換 =「目的変数の平均値を特徴量にする」
- ["curb_loc","steward","guards","sidewalk","user_type","borocode"]をターゲットエンコーディングする
→効果なし。他のすべての変数をターゲットエンコーディングしてみる
["nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]をターゲットエンコーディング



木の健康状態（目的変数）
0:Fair（普通） 1:Good（良い） 2:Poor（悪い）
0:良い、1:普通、2:悪い、という値に置き換える

→ミクロF!スコアが比較的高い分類モデルの混合行列を検討する。
LGBMのミクロF1スコアは0.7905929447085315

→比重付き精度(Balanced score)が比較的高い分類モデルの混合行列を検討する。
LGBMの比重付き精度は0.3471486196238671

NearestCentroid　0.49
GaussianNB　0.40
PassiveAggressiveClassifier	0.39
QuadraticDiscriminantAnalysis 0.39
ExtraTreeClassifier	0.38
DecisionTreeClassifier　0.37
LogisticRegression　0.36



全ての変数をターゲットエンコーディングしたところ、ようやく精度が向上した。
一見木とか関係なさそうな 州議会地区番号・市議会の地区番号が実が重要、ということが分かる。

In [ ]:
 ["tree_dbh","curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]
y_list = ["health"]

In [ ]:
	submit_randomforest_01.csv	0.3108493	木の分析、ターゲットエンコーディング +...修正	2023-12-14 10:08:23
	submit_randomforest_01.csv	0.1165707	木の分析、ランダムフォレスト法による予測修正	2023-12-13 14:33:06
	submit_gnb_01.csv	0.3415307	木の分析、ガウシアンナイーブベイズによる...修正	2023-12-13 14:27:36
	submit_lgbm_02.csv	0.2957859	木の分析、LGBMの改良版修正	2023-12-13 14:13:35
	submit_lgbm_nearcenter01.csv	0.3248569	木の分析、近傍中心法を使った予測修正	2023-12-13 14:06:32
	submit_lgbm_logreg01.csv	0.3045470	木の分析、ロジスティック回帰を使った予測修正	2023-12-13 14:04:06
	submit_lgbm_baseline01.csv	0.2943655	LGBMによるベースライン、6変数のみに...

In [ ]:
精度が高くなる予測モデルの特徴
→ Accuracyが過学習していない(0.79にならない、混合行列を見た際の再現度が高い)、
ミクロF1スコアが高い、

GaussianNB
マクロf1スコア：
0.3803493338446983

PassiveAggressiveClassifier
マクロf1スコア：
0.4168970929151401

ExtraTreeClassifier
マクロf1スコア：
0.37019650218319544

DecisionTreeClassifier
マクロf1スコア：
0.37882986018366616

QuadraticDiscriminantAnalysis
マクロf1スコア：
0.3720410136832782

LinearDiscriminantAnalysis
マクロf1スコア：
0.3581063198509942

BaggingClassifier
マクロf1スコア：
0.3713729527219161

Perceptron
マクロf1スコア：
0.35592729143234053

ExtraTreesClassifier
マクロf1スコア：
0.350071925186423

KNeighborsClassifier	

In [ ]:
LGBMで精度向上 → 最適化時の指標をマクロF1スコアにする
指標をマクロF1にするだけではこれ以上改善できない。
他のアルゴリズムを試したら特徴量エンジニアリングによる精度向上に
移行する

→ラベルが実際１・2の場合どのようなスコアが多いか、という特徴を
　分析する。ダミー化等でそうした指標を狙って値に入れる

In [ ]:
次元削減による精度向上 → LLE、PCA、TSNEなどの次元圧縮で
特徴量が発見しにくくなる「次元の呪い」を解く

In [ ]:
暫定評価が高いモデルでアンサンブル学習する
→ ガウシアンナイーブベイズ

In [ ]:
PassiveAggressiveClassifier
GaussianNB

In [ ]:
ターゲットエンコーディングに加え、
ユニーク数が少ない質的変数のダミー、
problemsの有無を含めたダミーを入れて
lazypredict時の値が向上するか検証する！

In [ ]:
次元圧縮 → うまく特徴量を抽出できていれば
t-SNEで圧縮した時にクラスターが浮かび上がるはず

In [ ]:
t-SNEで圧縮したデータを学習させて精度を検証する
→逆効果。16次元から2次元の圧縮では効果は薄い

In [ ]:
curb_loc, steward, guards, sidewalk, user_typeなど
木の状態に関係する値をダミー化する
problemsのみ、そのブール値をダミー化する

bool_problems

In [ ]:
効果のあった特徴量エンジニアリング・予測モデル

・ダミー化 → 効果なし
・ターゲットエンコーディング → 質的変数が多い場合に有効
　リーク情報を含めると精度が向上する
・木の種類→種類からより分析に有効な値を抽出
 
・LazyPredict → F1の高いモデルが有効

・木ダメージ=特徴量を抽出すればより精度を高めることができる
・時系列データ＝木ダメージと関係あるかも？
・"Problems" - どう処理するか？→問題が多いほどダメージがある

0：良い、1：普通、2：悪い

In [ ]:
GaussianNB

PassiveAggressiveClassifier

LinearDiscriminantAnalysis

ExtraTreesClassifier

BaggingClassifier

ExtraTreeClassifier